In [1]:
from torchsummary import summary
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from torchvision import models
import torch
from torch import nn

In [2]:
from model import AttNet
net1 = AttNet(1,15).cuda()

summary(net1, input_size=(1, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 512, 512]              16
            Conv2d-2         [-1, 16, 514, 514]              16
            Conv2d-3         [-1, 16, 514, 514]              16
     AttentionConv-4         [-1, 16, 512, 512]               0
       BatchNorm2d-5         [-1, 16, 512, 512]              32
              ReLU-6         [-1, 16, 512, 512]               0
            Conv2d-7         [-1, 16, 512, 512]             256
            Conv2d-8         [-1, 16, 514, 514]             256
            Conv2d-9         [-1, 16, 514, 514]             256
    AttentionConv-10         [-1, 16, 512, 512]               0
      BatchNorm2d-11         [-1, 16, 512, 512]              32
             ReLU-12         [-1, 16, 512, 512]               0
        MaxPool2d-13         [-1, 16, 256, 256]               0
           Conv2d-14         [-1, 32, 2

In [6]:


net2 = models.resnet18(pretrained=False)
net2.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = net2.fc.in_features
net2.fc = nn.Linear(num_ftrs, 15)
net = net2.cuda()
summary(net, input_size=(1, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           3,136
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1

In [12]:
net = models.resnet50(pretrained=False)
net.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 15)
net = net.cuda()
summary(net, input_size=(1, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           3,136
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]           4,096
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
           Conv2d-11        [-1, 256, 128, 128]          16,384
      BatchNorm2d-12        [-1, 256, 128, 128]             512
           Conv2d-13        [-1, 256, 128, 128]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [10]:
print("%.2f MB" %(os.path.getsize('/home/rakshith/miccai_2022/model_weights/classification/ssa_adam_1e4_e15_bce/model_best.pt')/1e6))

18.30 MB


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity

In [15]:
inputs = torch.randn(1,1,512,512).cuda()
net = net.cuda()
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        net(inputs)

In [16]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        20.36%       8.093ms        99.92%      39.717ms      39.717ms             1  
                                           aten::conv2d         5.29%       2.103ms        37.26%      14.809ms     279.415us            53  
                                      aten::convolution         1.64%     653.000us        31.96%      12.706ms     239.736us            53  
                                     aten::_convolution         2.16%     858.000us        30.32%      12.053ms     227.415us            53  
      

In [5]:
# AttentionNet 
inputs = torch.randn(1,1,512,512)
net1 = net1.cpu()
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        net(inputs)
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        11.04%     149.548ms       100.00%        1.354s        1.354s             1  
                     aten::conv2d         0.03%     474.000us        23.45%     317.598ms       9.624ms            33  
                aten::convolution         0.03%     410.000us        23.42%     317.124ms       9.610ms            33  
               aten::_convolution         1.74%      23.584ms        23.38%     316.714ms       9.597ms            33  
                        aten::cat         0.02%     318.000us        22.78%     308.456ms      28.041ms            11  
                       aten::_cat       

In [7]:
inputs = torch.randn(1,1,512,512)
net2 = net2.cpu()
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        net2(inputs)
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         0.85%       4.461ms        99.99%     527.354ms     527.354ms             1  
                     aten::conv2d         0.04%     217.000us        83.83%     442.119ms      22.106ms            20  
                aten::convolution         0.05%     265.000us        83.79%     441.902ms      22.095ms            20  
               aten::_convolution         0.08%     423.000us        83.74%     441.637ms      22.082ms            20  
         aten::mkldnn_convolution        83.62%     441.002ms        83.66%     441.214ms      22.061ms            20  
                 aten::max_pool2d       